In [3]:
import datasets
import numpy as np
import re
import spacy
import torch
import transformers
from tqdm.notebook import tqdm

In [5]:

# ex = transformers.pipeline(
#     task="text2text-generation", 
#     model="google/flan-t5-xxl", 
#     device=0,
#     torch_dtype=torch.bfloat16
# )

In [6]:

tok = transformers.AutoTokenizer.from_pretrained("huggyllama/llama-13b")
nlp = spacy.load("en_core_web_sm")

ds = datasets.load_dataset("aqua_rat", split="train")
def clean(sample):
    sample["rationale"] = sample["rationale"].replace("\n", " ")
    return sample
ds = ds.map(clean)


No config specified, defaulting to: aqua_rat/raw
Found cached dataset aqua_rat (/home/mila/g/gagnonju/.cache/huggingface/datasets/aqua_rat/raw/0.0.0/fc47b9f437236ab96fc1fcb61096aa193819aedd76437893e2390ab0740a3381)
Loading cached processed dataset at /home/mila/g/gagnonju/.cache/huggingface/datasets/aqua_rat/raw/0.0.0/fc47b9f437236ab96fc1fcb61096aa193819aedd76437893e2390ab0740a3381/cache-11f6ac031c8e3ef3.arrow


In [7]:
dl = torch.utils.data.DataLoader(ds, batch_size=64, collate_fn=None)

print({k: type(v) for k, v in next(iter(dl)).items()})

def ai_extractor(samples):
    batch = [
    "### Instructions:\n"
    "Extract the letter A, B, C, D, E "
    "from the following answer to a multiple choice question.\n"
    "### Answer to parse:\n"
    + sample for sample in samples]
    return [x["generated_text"] for x in ex(batch, do_sample=False)]

tqdm_obj = tqdm(dl)
output = []
parse_is_good = []
EXPECTED = ["A", "B", "C", "D", "E"]
for i, batch in enumerate(tqdm_obj):
    extracted = dumb_parser(batch["rationale"])
    output.extend([e == c for e, c in zip(extracted, batch["correct"])])
    if i <= 1:
        # print(f"{extracted        = }")
        # print(f"{batch['correct'] = }")
        for e, c, r in zip(extracted, batch['correct'], batch['rationale']):
            if e != c:
                print(f"{e = }")
                print(f"{c = }")
                print(f"{r = }")
                print("====")
    parse_is_good.extend([e in EXPECTED for e in extracted])
    tqdm_obj.set_description(f"Accuracy: {np.mean(output):0.1%}, Expected: {np.mean(parse_is_good):0.1%}")

{'question': <class 'list'>, 'options': <class 'list'>, 'rationale': <class 'list'>, 'correct': <class 'list'>}


  0%|          | 0/1523 [00:00<?, ?it/s]

NameError: name 'dumb_parser' is not defined

In [ ]:
def rindex(l, thing, default=None):
    if isinstance(l, str):
        try: 
            index = l.rindex(thing)
        except ValueError as err:
            if str(err) == "substring not found":
                return default
            raise
        return index

    for index, item in enumerate(reversed(l)):
        if item == thing:
            return len(l) - index - 1
            
    return default


class SpacyMultipleAnswerExtractor:
    def __init__(self, choices):
        self._choices = choices
        self._spacy = spacy.load("en_core_web_sm")
    
    def spacy_parse_one(self, text):
        text = text.replace(".", " ")
        pairs = []
        tokenized = list(map(str, self._spacy(text)))
        for choice in self._choices:
            pairs.append((choice, rindex(tokenized, choice, -1)))
        return max(pairs, key=lambda s: s[1])[0]
    
    def spacy_parse(self, batch):
        return [self.spacy_parse_one(x) for x in batch]

    __call__ = spacy_parse


spacy_parser = SpacyMultipleAnswerExtractor(EXPECTED)


class DumbestExtractor:
    def __init__(self, choices):
        self._choices = choices
    
    def spacy_parse_one(self, text):
        pairs = []
        for choice in self._choices:
            pairs.append((choice, rindex(text, choice, -1)))
        return max(pairs, key=lambda s: s[1])[0]
    
    def spacy_parse(self, batch):
        return [self.spacy_parse_one(x) for x in batch]

    __call__ = spacy_parse

dumb_parser = DumbestExtractor(EXPECTED)


class RegexExtractor:
    def __init__(self, choices):
        self._choices = choices
        self._pat = re.compile(r"\w+")

        # each choice should parse to a single thing,
        # otherwise the extractor will break
        test_pat = [self._pat.findall(c) for c in choices]
        assert all(len(l) == 1 for l in test_pat), test_pat

    def parse_one(self, text):
        text = self._pat.findall(text)
        pairs = []
        for choice in self._choices:
            pairs.append((choice, rindex(text, choice, -1)))
        return max(pairs, key=lambda s: s[1])[0]
    
    def parse(self, batch):
        return [self.parse_one(x) for x in batch]

    __call__ = parse

regex_extractor = RegexExtractor(EXPECTED)

NameError: name 'EXPECTED' is not defined

In [ ]:
tok.tokenize("EE")

['▁E', 'E']

In [ ]:
try:
    "abcdefg".rindex("1")
except ValueError as err:
    print(str(err) == "substring not found")
    raise

True


ValueError: substring not found

In [14]:
def rindex(l, thing, default=None):
    if isinstance(l, str):
        try: 
            index = l.rindex(thing)
        except ValueError as err:
            if str(err) == "substring not found":
                return default
            raise
        return index

    for index, item in enumerate(reversed(l)):
        if item == thing:
            return len(l) - index - 1
            
    return default

text = "asda23sdasdaddd23dsadasd"
thing = "55"
idx = rindex(text, thing)
print(idx)
text[idx: idx + len(thing)]

None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'